In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import csv

# Set up Selenium WebDriver (make sure to specify the path to your WebDriver if needed)
driver = webdriver.Chrome()  # or `webdriver.Firefox()` for Firefox

# Function to extract details from the current page
def extract_details():
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract the venue name
    venue_name_element = soup.find('p', id='slide-over-heading')
    venue_name = venue_name_element.text.strip() if venue_name_element else 'N/A'

    # Extract the address
    address_element = soup.find('p', class_='font-helvetica font-light text-green-dark text-[16px] tracking-[0] leading-[normal]')
    address = address_element.text.strip() if address_element else 'N/A'

    # Extract the email
    email_element = soup.find('a', class_='font-helvetica font-light text-green-dark text-[16px] tracking-[0] leading-[normal]', href=lambda href: href and "mailto:" in href)
    email = email_element.get('href').replace('mailto:', '').strip() if email_element else 'N/A'

    # Extract the phone number (if needed)
    phone_element = soup.find('a', href=lambda href: href and "tel:" in href)
    phone = phone_element.get('href').replace('tel:', '').strip() if phone_element else 'N/A'

    return [venue_name, address, email, phone]

# Function to close the details panel
def close_details_panel():
    try:
        close_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@class,"relative") and contains(@class,"rounded-md") and contains(@class,"bg-white") and contains(@class,"text-gray-400")]'))
        )
        driver.execute_script("arguments[0].click();", close_button)  # Click using JavaScript
        print("Closed the details popup.")
        time.sleep(1)  # Wait for the popup to close
    except Exception as e:
        print(f"Unable to close the details popup: {e}")

# Function to get the total number of "Show Details" buttons
def get_show_details_buttons_count():
    buttons = driver.find_elements(By.XPATH, '//button[contains(@class,"rounded-full") and contains(text(),"Show Details")]')
    return len(buttons)

# Function to process a single URL
def process_url(url, handle_cookies):
    driver.get(url)
    
    # Handle cookies pop-up if needed
    if handle_cookies:
        try:
            accept_cookies_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//button[text()="Accept"]'))
            )
            accept_cookies_button.click()
            print("Cookies pop-up accepted.")
        except Exception as e:
            print(f"No cookies pop-up found or unable to click: {e}")

    total_buttons = get_show_details_buttons_count()
    half_buttons = total_buttons // 2
    print(f"Found {total_buttons} 'Show Details' buttons. Processing {half_buttons}.")

    for i in range(half_buttons):
        try:
            # Find all "Show Details" buttons on the page
            buttons = driver.find_elements(By.XPATH, '//button[contains(@class,"rounded-full") and contains(text(),"Show Details")]')
            
            if i >= len(buttons):
                print("All buttons processed.")
                break

            button = buttons[i]
            driver.execute_script("arguments[0].scrollIntoView();", button)  # Scroll to the button
            driver.execute_script("arguments[0].click();", button)  # Click using JavaScript
            print(f"Clicked on 'Show Details' button {i + 1}.")
            
            # Wait for the details to be visible
            time.sleep(3)
            
            # Extract details after clicking the button
            details = extract_details()
            
            # Write the details to the CSV file
            csv_writer.writerow(details)
            
            # Close the details popup
            close_details_panel()
            
            # Re-fetch the buttons list and total count after closing the popup to update the count
            total_buttons = get_show_details_buttons_count()
            half_buttons = total_buttons // 2
            print(f"Updated total count of 'Show Details' buttons: {total_buttons}. Processing {half_buttons}.")

        except Exception as e:
            print(f"Error while processing button {i + 1}: {e}")
            driver.save_screenshot(f'error_screenshot_{i + 1}.png')  # Save a screenshot for debugging

# Open a CSV file to write the results
csv_file = open('details_output.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)

# Write the headers to the CSV file
csv_writer.writerow(['Venue Name', 'Address', 'Email', 'Phone'])

# List of URLs to process
urls = [
    ('https://screengolfers.com/usa/colorado/denver', False),
('https://screengolfers.com/usa/colorado/colorado-springs', False),
('https://screengolfers.com/usa/colorado/englewood', False),
('https://screengolfers.com/usa/colorado/fort-collins', False),
('https://screengolfers.com/usa/colorado/grand-junction', False),
('https://screengolfers.com/usa/colorado/highlands-ranch', False),
('https://screengolfers.com/usa/colorado/parker', False),
('https://screengolfers.com/usa/colorado/westminster', False),
('https://screengolfers.com/usa/colorado/avon', False),
('https://screengolfers.com/usa/colorado/centennial', False),
('https://screengolfers.com/usa/colorado/golden', False),
('https://screengolfers.com/usa/colorado/greenwood-village', False),
('https://screengolfers.com/usa/colorado/lakewood', False),
('https://screengolfers.com/usa/colorado/littleton', False),
('https://screengolfers.com/usa/colorado/lone-tree', False),
('https://screengolfers.com/usa/colorado/loveland', False),
('https://screengolfers.com/usa/colorado/superior', False),
('https://screengolfers.com/usa/colorado/thornton', False),
('https://screengolfers.com/usa/delaware/newark', False),
('https://screengolfers.com/usa/delaware/ocean-view', False),
('https://screengolfers.com/usa/delaware/wilmington', False),
('https://screengolfers.com/usa/missouri/st-louis', False),
('https://screengolfers.com/usa/missouri/chesterfield', False),
('https://screengolfers.com/usa/missouri/joplin', False),
('https://screengolfers.com/usa/missouri/brookline', False),
('https://screengolfers.com/usa/missouri/ellisville', False),
('https://screengolfers.com/usa/missouri/lees-summit', False),
('https://screengolfers.com/usa/missouri/nixa', False),
('https://screengolfers.com/usa/missouri/ozark', False),
('https://screengolfers.com/usa/missouri/parkville', False),
('https://screengolfers.com/usa/missouri/saint-louis', False),
('https://screengolfers.com/usa/missouri/springfield', False),
('https://screengolfers.com/usa/missouri/warrensburg', False),
('https://screengolfers.com/usa/montana/kalispell', False),
('https://screengolfers.com/usa/montana/billings', False),
('https://screengolfers.com/usa/montana/bozeman', False),
('https://screengolfers.com/usa/montana/cut-bank', False),
('https://screengolfers.com/usa/montana/great-falls', False),
('https://screengolfers.com/usa/montana/lewistown', False),
('https://screengolfers.com/usa/pennsylvania/philadelphia', False),
('https://screengolfers.com/usa/pennsylvania/pittsburgh', False),
('https://screengolfers.com/usa/pennsylvania/allentown', False),
('https://screengolfers.com/usa/pennsylvania/harrisburg', False),
('https://screengolfers.com/usa/pennsylvania/wexford', False),
('https://screengolfers.com/usa/pennsylvania/ardmore', False),
('https://screengolfers.com/usa/pennsylvania/berwyn', False),
('https://screengolfers.com/usa/pennsylvania/chester-springs', False),
('https://screengolfers.com/usa/pennsylvania/clarion', False),
('https://screengolfers.com/usa/pennsylvania/conshohocken', False),
('https://screengolfers.com/usa/pennsylvania/dalton', False),
('https://screengolfers.com/usa/pennsylvania/easton', False),
('https://screengolfers.com/usa/pennsylvania/ellwood-city', False),
('https://screengolfers.com/usa/pennsylvania/erie', False),
('https://screengolfers.com/usa/pennsylvania/gettysburg', False),
('https://screengolfers.com/usa/pennsylvania/hanover', False),
('https://screengolfers.com/usa/pennsylvania/hatfield', False),
('https://screengolfers.com/usa/pennsylvania/hershey', False),
('https://screengolfers.com/usa/pennsylvania/jeannette', False),
('https://screengolfers.com/usa/pennsylvania/jessup', False),
('https://screengolfers.com/usa/pennsylvania/lancaster', False),
('https://screengolfers.com/usa/pennsylvania/langhorne', False),
('https://screengolfers.com/usa/pennsylvania/latrobe', False),
('https://screengolfers.com/usa/pennsylvania/lititz', False),
('https://screengolfers.com/usa/pennsylvania/malvern', False),
('https://screengolfers.com/usa/pennsylvania/mars', False),
('https://screengolfers.com/usa/pennsylvania/mechanicsburg', False),
('https://screengolfers.com/usa/pennsylvania/newton', False),
('https://screengolfers.com/usa/pennsylvania/north-huntingdon', False),
('https://screengolfers.com/usa/pennsylvania/north-wales', False),
('https://screengolfers.com/usa/pennsylvania/oakmont', False),
('https://screengolfers.com/usa/pennsylvania/phoenixville', False),
('https://screengolfers.com/usa/pennsylvania/punxsutawney', False),
('https://screengolfers.com/usa/pennsylvania/royersford', False),
('https://screengolfers.com/usa/pennsylvania/scott-township', False),
('https://screengolfers.com/usa/pennsylvania/springfield', False),
('https://screengolfers.com/usa/pennsylvania/stewartstown', False),
('https://screengolfers.com/usa/pennsylvania/warren', False),
('https://screengolfers.com/usa/pennsylvania/west-chester', False),
('https://screengolfers.com/usa/pennsylvania/west-norriton', False),
('https://screengolfers.com/usa/pennsylvania/wilkes-barre', False),
('https://screengolfers.com/usa/pennsylvania/willow-grove', False),
('https://screengolfers.com/usa/pennsylvania/york', False),
('https://screengolfers.com/usa/south-carolina/greenville', False),
('https://screengolfers.com/usa/south-carolina/mount-pleasant', False),
('https://screengolfers.com/usa/south-carolina/bluffton', False),
('https://screengolfers.com/usa/south-carolina/charleston', False),
('https://screengolfers.com/usa/south-carolina/florence', False),
('https://screengolfers.com/usa/south-carolina/fort-mill', False),
('https://screengolfers.com/usa/south-carolina/hilton-head', False),
('https://screengolfers.com/usa/south-carolina/hilton-head-island', False),
('https://screengolfers.com/usa/south-carolina/lexington', False),
('https://screengolfers.com/usa/south-carolina/myrtle-beach', False),
('https://screengolfers.com/usa/wisconsin/appleton', False),
('https://screengolfers.com/usa/wisconsin/brookfield', False),
('https://screengolfers.com/usa/wisconsin/madison', False),
('https://screengolfers.com/usa/wisconsin/middleton', False),
('https://screengolfers.com/usa/wisconsin/beloit', False),
('https://screengolfers.com/usa/wisconsin/east-troy', False),
('https://screengolfers.com/usa/wisconsin/germantown', False),
('https://screengolfers.com/usa/wisconsin/green-bay', False),
('https://screengolfers.com/usa/wisconsin/greendale', False),
('https://screengolfers.com/usa/wisconsin/janesville', False),
('https://screengolfers.com/usa/wisconsin/kenosha', False),
('https://screengolfers.com/usa/wisconsin/mequon', False),
('https://screengolfers.com/usa/wisconsin/milwaukee', False),
('https://screengolfers.com/usa/wisconsin/rice-lake', False),
('https://screengolfers.com/usa/wisconsin/south-milwaukee', False),
('https://screengolfers.com/usa/wisconsin/thiensville', False),
('https://screengolfers.com/usa/wisconsin/waupun', False),
('https://screengolfers.com/usa/wisconsin/wausau', False),
('https://screengolfers.com/usa/vermont/south-burlington', False),
('https://screengolfers.com/usa/vermont/rutland', False),
('https://screengolfers.com/usa/maryland/annapolis', False),
('https://screengolfers.com/usa/maryland/baltimore', False),
('https://screengolfers.com/usa/maryland/ellicott-city', False),
('https://screengolfers.com/usa/maryland/bel-air', False),
('https://screengolfers.com/usa/maryland/churchville', False),
('https://screengolfers.com/usa/maryland/columbia', False),
('https://screengolfers.com/usa/maryland/easton', False),
('https://screengolfers.com/usa/maryland/edgewater', False),
('https://screengolfers.com/usa/maryland/elkridge', False),
('https://screengolfers.com/usa/maryland/forest-hill', False),
('https://screengolfers.com/usa/maryland/frederick', False),
('https://screengolfers.com/usa/maryland/gaithersburg', False),
('https://screengolfers.com/usa/maryland/hunt-valley', False),
('https://screengolfers.com/usa/maryland/leonardtown', False),
('https://screengolfers.com/usa/maryland/lutherville-timonium', False),
('https://screengolfers.com/usa/maryland/middletown', False),
('https://screengolfers.com/usa/maryland/north-bethesda', False),
('https://screengolfers.com/usa/maryland/owings-mills', False),
('https://screengolfers.com/usa/maryland/pasadena', False),
('https://screengolfers.com/usa/maryland/pikesville', False),
('https://screengolfers.com/usa/maryland/rockville', False),
('https://screengolfers.com/usa/maryland/towson', False),
('https://screengolfers.com/usa/minnesota/minneapolis', False),
('https://screengolfers.com/usa/minnesota/saint-paul', False),
('https://screengolfers.com/usa/minnesota/eden-prairie', False),
('https://screengolfers.com/usa/minnesota/lakeville', False),
('https://screengolfers.com/usa/minnesota/rochester', False),
('https://screengolfers.com/usa/minnesota/burnsville', False),
('https://screengolfers.com/usa/minnesota/apple-valley', False),
('https://screengolfers.com/usa/minnesota/champlin', False),
('https://screengolfers.com/usa/minnesota/detroit-lakes', False),
('https://screengolfers.com/usa/minnesota/duluth', False),
('https://screengolfers.com/usa/minnesota/eagan', False),
('https://screengolfers.com/usa/minnesota/excelsior', False),
('https://screengolfers.com/usa/minnesota/faribault', False),
('https://screengolfers.com/usa/minnesota/fergus-falls', False),
('https://screengolfers.com/usa/minnesota/fertile', False),
('https://screengolfers.com/usa/minnesota/hamel', False),
('https://screengolfers.com/usa/minnesota/long-lake', False),
('https://screengolfers.com/usa/minnesota/mankato', False),
('https://screengolfers.com/usa/minnesota/maple-grove', False),
('https://screengolfers.com/usa/minnesota/moorhead', False),
('https://screengolfers.com/usa/minnesota/new-ulm', False),
('https://screengolfers.com/usa/minnesota/osakis', False),
('https://screengolfers.com/usa/minnesota/plymouth', False),
('https://screengolfers.com/usa/minnesota/prior-lake', False),
('https://screengolfers.com/usa/minnesota/richfield', False),
('https://screengolfers.com/usa/minnesota/roseville', False),
('https://screengolfers.com/usa/minnesota/saint-james', False),
('https://screengolfers.com/usa/minnesota/slayton', False),
('https://screengolfers.com/usa/minnesota/willmar', False),
('https://screengolfers.com/usa/minnesota/woodbury', False),
('https://screengolfers.com/usa/virginia/fairfax', False),
('https://screengolfers.com/usa/virginia/glen-allen', False),
('https://screengolfers.com/usa/virginia/midlothian', False),
('https://screengolfers.com/usa/virginia/virginia-beach', False),
('https://screengolfers.com/usa/virginia/alexandria', False),
('https://screengolfers.com/usa/virginia/ashburn', False),
('https://screengolfers.com/usa/virginia/chesapeake', False),
('https://screengolfers.com/usa/virginia/falls-church', False),
('https://screengolfers.com/usa/virginia/fredericksburg', False),
('https://screengolfers.com/usa/virginia/herndon', False),
('https://screengolfers.com/usa/virginia/leesburg', False),
('https://screengolfers.com/usa/virginia/mclean', False),
('https://screengolfers.com/usa/massachusetts/burlington', False),
('https://screengolfers.com/usa/massachusetts/worcester', False),
('https://screengolfers.com/usa/massachusetts/boston', False),
('https://screengolfers.com/usa/massachusetts/fitchburg', False),
('https://screengolfers.com/usa/massachusetts/hanover', False),
('https://screengolfers.com/usa/massachusetts/waltham', False),
('https://screengolfers.com/usa/massachusetts/westfield', False),
('https://screengolfers.com/usa/massachusetts/acton', False),
('https://screengolfers.com/usa/massachusetts/ashland', False),
('https://screengolfers.com/usa/massachusetts/auburn', False),
('https://screengolfers.com/usa/massachusetts/bellingham', False),
('https://screengolfers.com/usa/massachusetts/cambridge', False),
('https://screengolfers.com/usa/massachusetts/dalton', False),
('https://screengolfers.com/usa/massachusetts/dudley', False),
('https://screengolfers.com/usa/massachusetts/easthampton', False),
('https://screengolfers.com/usa/massachusetts/east-walpole', False),
('https://screengolfers.com/usa/massachusetts/falmouth', False),
('https://screengolfers.com/usa/massachusetts/framingham', False),
('https://screengolfers.com/usa/massachusetts/franklin', False),
('https://screengolfers.com/usa/massachusetts/halifax', False),
('https://screengolfers.com/usa/massachusetts/jefferson', False),
('https://screengolfers.com/usa/massachusetts/lakeville', False),
('https://screengolfers.com/usa/massachusetts/littleton', False),
('https://screengolfers.com/usa/massachusetts/methuen', False),
('https://screengolfers.com/usa/massachusetts/middleton', False),
('https://screengolfers.com/usa/massachusetts/natick', False),
('https://screengolfers.com/usa/massachusetts/needham-heights', False),
('https://screengolfers.com/usa/massachusetts/north-chelmsford', False),
('https://screengolfers.com/usa/massachusetts/orleans', False),
('https://screengolfers.com/usa/massachusetts/peabody', False),
('https://screengolfers.com/usa/massachusetts/plymouth', False),
('https://screengolfers.com/usa/massachusetts/reading', False),
('https://screengolfers.com/usa/massachusetts/south-easton', False),
('https://screengolfers.com/usa/massachusetts/springfield', False),
('https://screengolfers.com/usa/massachusetts/wayland', False),
('https://screengolfers.com/usa/massachusetts/westborough', False),
('https://screengolfers.com/usa/massachusetts/west-boylston', False),
('https://screengolfers.com/usa/massachusetts/westford', False),
('https://screengolfers.com/usa/massachusetts/woburn', False),
('https://screengolfers.com/usa/nevada/las-vegas', False),
('https://screengolfers.com/usa/nevada/reno', False),
('https://screengolfers.com/usa/nevada/henderson', False),
('https://screengolfers.com/usa/michigan/grand-rapids', False),
('https://screengolfers.com/usa/michigan/ann-arbor', False),
('https://screengolfers.com/usa/michigan/livonia', False),
('https://screengolfers.com/usa/michigan/novi', False),
('https://screengolfers.com/usa/michigan/troy', False),
('https://screengolfers.com/usa/michigan/brighton', False),
('https://screengolfers.com/usa/michigan/clinton-township', False),
('https://screengolfers.com/usa/michigan/detroit', False),
('https://screengolfers.com/usa/michigan/grandville', False),
('https://screengolfers.com/usa/michigan/holland', False),
('https://screengolfers.com/usa/michigan/kalamazoo', False),
('https://screengolfers.com/usa/michigan/linden', False),
('https://screengolfers.com/usa/michigan/rochester-hills', False),
('https://screengolfers.com/usa/michigan/shelby-township', False),
('https://screengolfers.com/usa/michigan/adrian', False),
('https://screengolfers.com/usa/michigan/auburn-hills', False),
('https://screengolfers.com/usa/michigan/benton-harbor', False),
('https://screengolfers.com/usa/michigan/canton', False),
('https://screengolfers.com/usa/michigan/coldwater', False),
('https://screengolfers.com/usa/michigan/commerce-township', False),
('https://screengolfers.com/usa/michigan/comstock-park', False),
('https://screengolfers.com/usa/michigan/dewitt', False),
('https://screengolfers.com/usa/michigan/howell', False),
('https://screengolfers.com/usa/michigan/kimball', False),
('https://screengolfers.com/usa/michigan/ludington', False),
('https://screengolfers.com/usa/michigan/macomb', False),
('https://screengolfers.com/usa/michigan/midland', False),
('https://screengolfers.com/usa/michigan/muskegon', False),
('https://screengolfers.com/usa/michigan/new-lothrop', False),
('https://screengolfers.com/usa/michigan/norton-shores', False),
('https://screengolfers.com/usa/michigan/okemos', False),
('https://screengolfers.com/usa/michigan/oxford', False),
('https://screengolfers.com/usa/michigan/petoskey', False),
('https://screengolfers.com/usa/michigan/plymouth', False),
('https://screengolfers.com/usa/michigan/ravenna', False),
('https://screengolfers.com/usa/michigan/rockwood', False),
('https://screengolfers.com/usa/michigan/royal-oak', False),
('https://screengolfers.com/usa/michigan/schoolcraft', False),
('https://screengolfers.com/usa/michigan/st-clair-shores', False),
('https://screengolfers.com/usa/michigan/sturgis', False),
('https://screengolfers.com/usa/michigan/traverse-city', False),
('https://screengolfers.com/usa/michigan/waterford', False),
('https://screengolfers.com/usa/michigan/wayland', False),
('https://screengolfers.com/usa/michigan/woodhaven', False),
('https://screengolfers.com/usa/mississippi/biloxi', False),
('https://screengolfers.com/usa/florida/orlando', False),
('https://screengolfers.com/usa/florida/naples', False),
('https://screengolfers.com/usa/florida/tampa', False),
('https://screengolfers.com/usa/florida/clearwater', False),
('https://screengolfers.com/usa/florida/fort-lauderdale', False),
('https://screengolfers.com/usa/florida/sarasota', False),
('https://screengolfers.com/usa/florida/stuart', False),
('https://screengolfers.com/usa/florida/altamonte-springs', False),
('https://screengolfers.com/usa/florida/cape-coral', False),
('https://screengolfers.com/usa/florida/cocoa-beach', False),
('https://screengolfers.com/usa/florida/coconut-creek', False),
('https://screengolfers.com/usa/florida/coral-springs', False),
('https://screengolfers.com/usa/florida/estero', False),
('https://screengolfers.com/usa/florida/fort-myers', False),
('https://screengolfers.com/usa/florida/fort-walton-beach', False),
('https://screengolfers.com/usa/florida/havana', False),
('https://screengolfers.com/usa/florida/hudson', False),
('https://screengolfers.com/usa/florida/jacksonville', False),
('https://screengolfers.com/usa/florida/jacksonville-beach', False),
('https://screengolfers.com/usa/florida/jupiter', False),
('https://screengolfers.com/usa/florida/kissimmee', False),
('https://screengolfers.com/usa/florida/lakewood-ranch', False),
('https://screengolfers.com/usa/florida/largo', False),
('https://screengolfers.com/usa/florida/lauderdale-by-the-sea', False),
('https://screengolfers.com/usa/florida/lutz', False),
('https://screengolfers.com/usa/florida/maitland', False),
('https://screengolfers.com/usa/florida/palm-beach-gardens', False),
('https://screengolfers.com/usa/florida/pembroke-pines', False),
('https://screengolfers.com/usa/florida/pompano-beach', False),
('https://screengolfers.com/usa/florida/port-charlotte', False),
('https://screengolfers.com/usa/florida/port-st-lucie', False),
('https://screengolfers.com/usa/florida/saint-petersburg', False),
('https://screengolfers.com/usa/florida/west-palm-beach', False),
('https://screengolfers.com/usa/oregon/portland', False),
('https://screengolfers.com/usa/oregon/beaverton', False),
('https://screengolfers.com/usa/oregon/tualatin', False),
('https://screengolfers.com/usa/oregon/albany', False),
('https://screengolfers.com/usa/oregon/central-point', False),
('https://screengolfers.com/usa/oregon/gleneden-beach', False),
('https://screengolfers.com/usa/oregon/klamath-falls', False),
('https://screengolfers.com/usa/oregon/oregon-city', False),
('https://screengolfers.com/usa/oregon/terrebonne', False),
('https://screengolfers.com/usa/oklahoma/oklahoma-city', False),
('https://screengolfers.com/usa/oklahoma/edmond', False),
('https://screengolfers.com/usa/oklahoma/tulsa', False),
('https://screengolfers.com/usa/wyoming/casper', False),
('https://screengolfers.com/usa/wyoming/laramie', False),
('https://screengolfers.com/usa/wyoming/cheyenne', False),
('https://screengolfers.com/usa/south-dakota/rapid-city', False),
('https://screengolfers.com/usa/south-dakota/watertown', False),
('https://screengolfers.com/usa/south-dakota/brookings', False),
('https://screengolfers.com/usa/south-dakota/deadwood', False),
('https://screengolfers.com/usa/south-dakota/huron', False),
('https://screengolfers.com/usa/south-dakota/sioux-falls', False),
('https://screengolfers.com/usa/south-dakota/spearfish', False),
('https://screengolfers.com/usa/south-dakota/vermillion', False),
('https://screengolfers.com/usa/connecticut/fairfield', False),
('https://screengolfers.com/usa/connecticut/glastonbury', False),
('https://screengolfers.com/usa/connecticut/milford', False),
('https://screengolfers.com/usa/connecticut/stamford', False),
('https://screengolfers.com/usa/connecticut/trumbull', False),
('https://screengolfers.com/usa/connecticut/canton', False),
('https://screengolfers.com/usa/connecticut/danbury', False),
('https://screengolfers.com/usa/connecticut/danielson', False),
('https://screengolfers.com/usa/connecticut/essex', False),
('https://screengolfers.com/usa/connecticut/monroe', False),
('https://screengolfers.com/usa/connecticut/new-britain', False),
('https://screengolfers.com/usa/connecticut/newington', False),
('https://screengolfers.com/usa/connecticut/north-haven', False),
('https://screengolfers.com/usa/connecticut/norwalk', False),
('https://screengolfers.com/usa/connecticut/orange', False),
('https://screengolfers.com/usa/connecticut/portland', False),
('https://screengolfers.com/usa/connecticut/stratford', False),
('https://screengolfers.com/usa/connecticut/waterbury', False),
('https://screengolfers.com/usa/connecticut/west-hartford', False),
('https://screengolfers.com/usa/connecticut/westport', False),
('https://screengolfers.com/usa/west-virginia/bunker-hill', False),
('https://screengolfers.com/usa/west-virginia/clarksburg', False),
('https://screengolfers.com/usa/west-virginia/fairmont', False),
('https://screengolfers.com/usa/west-virginia/huntington', False),
('https://screengolfers.com/usa/west-virginia/peterstown', False),
('https://screengolfers.com/usa/west-virginia/princeton', False),
('https://screengolfers.com/usa/west-virginia/triadelphia', False),
('https://screengolfers.com/usa/texas/austin', False),
('https://screengolfers.com/usa/texas/houston', False),
('https://screengolfers.com/usa/texas/dallas', False),
('https://screengolfers.com/usa/texas/katy', False),
('https://screengolfers.com/usa/texas/grapevine', False),
('https://screengolfers.com/usa/texas/allen', False),
('https://screengolfers.com/usa/texas/arlington', False),
('https://screengolfers.com/usa/texas/carrollton', False),
('https://screengolfers.com/usa/texas/fort-worth', False),
('https://screengolfers.com/usa/texas/friendswood', False),
('https://screengolfers.com/usa/texas/frisco', False),
('https://screengolfers.com/usa/texas/san-antonio', False),
('https://screengolfers.com/usa/texas/southlake', False),
('https://screengolfers.com/usa/texas/addison', False),
('https://screengolfers.com/usa/texas/amarillo', False),
('https://screengolfers.com/usa/texas/college-station', False),
('https://screengolfers.com/usa/texas/conroe', False),
('https://screengolfers.com/usa/texas/cypress', False),
('https://screengolfers.com/usa/texas/irving', False),
('https://screengolfers.com/usa/texas/kemah', False),
('https://screengolfers.com/usa/texas/longview', False),
('https://screengolfers.com/usa/texas/lubbock', False),
('https://screengolfers.com/usa/texas/marble-falls', False),
('https://screengolfers.com/usa/texas/mckinney', False),
('https://screengolfers.com/usa/texas/plano', False),
('https://screengolfers.com/usa/texas/rockwall', False),
('https://screengolfers.com/usa/texas/round-rock', False),
('https://screengolfers.com/usa/texas/schertz', False),
('https://screengolfers.com/usa/texas/spring', False),
('https://screengolfers.com/usa/texas/sugar-land', False),
('https://screengolfers.com/usa/texas/tomball', False),
('https://screengolfers.com/usa/new-mexico/albuquerque', False),
('https://screengolfers.com/usa/new-mexico/carlsbad', False),
('https://screengolfers.com/usa/alabama/birmingham', False),
('https://screengolfers.com/usa/alabama/huntsville', False),
('https://screengolfers.com/usa/ohio/canton', False),
('https://screengolfers.com/usa/ohio/cleveland', False),
('https://screengolfers.com/usa/ohio/akron', False),
('https://screengolfers.com/usa/ohio/columbus', False),
('https://screengolfers.com/usa/ohio/dayton', False),
('https://screengolfers.com/usa/ohio/north-ridgeville', False),
('https://screengolfers.com/usa/ohio/avon', False),
('https://screengolfers.com/usa/ohio/blue-ash', False),
('https://screengolfers.com/usa/ohio/dublin', False),
('https://screengolfers.com/usa/ohio/holland', False),
('https://screengolfers.com/usa/ohio/north-olmsted', False),
('https://screengolfers.com/usa/ohio/toledo', False),
('https://screengolfers.com/usa/ohio/zanesville', False),
('https://screengolfers.com/usa/ohio/ashland', False),
('https://screengolfers.com/usa/ohio/beachwood', False),
('https://screengolfers.com/usa/ohio/bellefontaine', False),
('https://screengolfers.com/usa/ohio/brecksville', False),
('https://screengolfers.com/usa/ohio/broadview-heights', False),
('https://screengolfers.com/usa/ohio/celina', False),
('https://screengolfers.com/usa/ohio/chagrin-falls', False),
('https://screengolfers.com/usa/ohio/cincinnati', False),
('https://screengolfers.com/usa/ohio/columbia-station', False),
('https://screengolfers.com/usa/ohio/elyria', False),
('https://screengolfers.com/usa/ohio/fairlawn', False),
('https://screengolfers.com/usa/ohio/findlay', False),
('https://screengolfers.com/usa/ohio/grandview-heights', False),
('https://screengolfers.com/usa/ohio/harrison', False),
('https://screengolfers.com/usa/ohio/heath', False),
('https://screengolfers.com/usa/ohio/hilliard', False),
('https://screengolfers.com/usa/ohio/hinckley', False),
('https://screengolfers.com/usa/ohio/lima', False),
('https://screengolfers.com/usa/ohio/loveland', False),
('https://screengolfers.com/usa/ohio/mason', False),
('https://screengolfers.com/usa/ohio/medina', False),
('https://screengolfers.com/usa/ohio/miamisburg', False),
('https://screengolfers.com/usa/ohio/miamitown', False),
('https://screengolfers.com/usa/ohio/mount-vernon', False),
('https://screengolfers.com/usa/ohio/newark', False),
('https://screengolfers.com/usa/ohio/north-canton', False),
('https://screengolfers.com/usa/ohio/ontario', False),
('https://screengolfers.com/usa/ohio/perrysburg', False),
('https://screengolfers.com/usa/ohio/pickerington', False),
('https://screengolfers.com/usa/ohio/port-clinton', False),
('https://screengolfers.com/usa/ohio/powell', False),
('https://screengolfers.com/usa/ohio/radnor', False),
('https://screengolfers.com/usa/ohio/sandusky', False),
('https://screengolfers.com/usa/ohio/solon', False),
('https://screengolfers.com/usa/ohio/springdale', False),
('https://screengolfers.com/usa/ohio/stow', False),
('https://screengolfers.com/usa/ohio/sugarcreek', False),
('https://screengolfers.com/usa/north-carolina/raleigh', False),
('https://screengolfers.com/usa/north-carolina/cary', False),
('https://screengolfers.com/usa/north-carolina/greensboro', False),
('https://screengolfers.com/usa/north-carolina/charlotte', False),
('https://screengolfers.com/usa/north-carolina/pineville', False),
('https://screengolfers.com/usa/north-carolina/winston-salem', False),
('https://screengolfers.com/usa/north-carolina/advance', False),
('https://screengolfers.com/usa/north-carolina/angier', False),
('https://screengolfers.com/usa/north-carolina/durham', False),
('https://screengolfers.com/usa/north-carolina/huntersville', False),
('https://screengolfers.com/usa/north-carolina/kernersville', False),
('https://screengolfers.com/usa/north-carolina/kitty-hawk', False),
('https://screengolfers.com/usa/north-carolina/mooresville', False),
('https://screengolfers.com/usa/north-carolina/roxboro', False),
('https://screengolfers.com/usa/north-carolina/southern-pines', False),
('https://screengolfers.com/usa/north-carolina/southport', False),
('https://screengolfers.com/usa/north-carolina/wake-forest', False),
('https://screengolfers.com/usa/north-carolina/wilmington', False),
('https://screengolfers.com/usa/north-carolina/winterville', False),
('https://screengolfers.com/usa/district-of-columbia/washington', False),
('https://screengolfers.com/usa/georgia/atlanta', False),
('https://screengolfers.com/usa/georgia/savannah', False),
('https://screengolfers.com/usa/georgia/alpharetta', False),
('https://screengolfers.com/usa/georgia/augusta', False),
('https://screengolfers.com/usa/georgia/carrollton', False),
('https://screengolfers.com/usa/georgia/duluth', False),
('https://screengolfers.com/usa/georgia/gray', False),
('https://screengolfers.com/usa/georgia/homer', False),
('https://screengolfers.com/usa/georgia/lagrange', False),
('https://screengolfers.com/usa/georgia/milton', False),
('https://screengolfers.com/usa/georgia/sandy-springs', False),
('https://screengolfers.com/usa/georgia/smyrna', False),
('https://screengolfers.com/usa/georgia/suwanee', False),
('https://screengolfers.com/usa/indiana/fort-wayne', False),
('https://screengolfers.com/usa/indiana/indianapolis', False),
('https://screengolfers.com/usa/indiana/carmel', False),
('https://screengolfers.com/usa/indiana/plainfield', False),
('https://screengolfers.com/usa/indiana/castleton', False),
('https://screengolfers.com/usa/indiana/crown-point', False),
('https://screengolfers.com/usa/indiana/delphi', False),
('https://screengolfers.com/usa/indiana/dyer', False),
('https://screengolfers.com/usa/indiana/noblesville', False),
('https://screengolfers.com/usa/indiana/portage', False),
('https://screengolfers.com/usa/indiana/schererville', False),
('https://screengolfers.com/usa/indiana/valparaiso', False),
('https://screengolfers.com/usa/indiana/warsaw', False),
('https://screengolfers.com/usa/indiana/washington', False),
('https://screengolfers.com/usa/indiana/zionsville', False),
('https://screengolfers.com/usa/rhode-island/warwick', False),
('https://screengolfers.com/usa/rhode-island/cranston', False),
('https://screengolfers.com/usa/rhode-island/east-greenwich', False),
('https://screengolfers.com/usa/rhode-island/east-providence', False),
('https://screengolfers.com/usa/rhode-island/lincoln', False),
('https://screengolfers.com/usa/rhode-island/north-kingstown', False),
('https://screengolfers.com/usa/rhode-island/portsmouth', False),
('https://screengolfers.com/usa/rhode-island/providence', False),
('https://screengolfers.com/usa/rhode-island/westerly', False),
('https://screengolfers.com/usa/kansas/overland-park', False),
('https://screengolfers.com/usa/kansas/topeka', False),
('https://screengolfers.com/usa/kansas/wichita', False),
('https://screengolfers.com/usa/kansas/bonner-springs', False),
('https://screengolfers.com/usa/kansas/kansas-city', False),
('https://screengolfers.com/usa/kansas/leawood', False),
('https://screengolfers.com/usa/kansas/olathe', False),
('https://screengolfers.com/usa/kansas/shawnee', False),
('https://screengolfers.com/usa/new-york/new-york-city', False),
('https://screengolfers.com/usa/new-york/white-plains', False),
('https://screengolfers.com/usa/new-york/amherst', False),
('https://screengolfers.com/usa/new-york/bay-shore', False),
('https://screengolfers.com/usa/new-york/bayside', False),
('https://screengolfers.com/usa/new-york/brooklyn', False),
('https://screengolfers.com/usa/new-york/carle-place', False),
('https://screengolfers.com/usa/new-york/centereach', False),
('https://screengolfers.com/usa/new-york/college-point', False),
('https://screengolfers.com/usa/new-york/commack', False),
('https://screengolfers.com/usa/new-york/dewitt', False),
('https://screengolfers.com/usa/new-york/huntington', False),
('https://screengolfers.com/usa/new-york/long-island-city', False),
('https://screengolfers.com/usa/new-york/queens', False),
('https://screengolfers.com/usa/new-york/rochester', False),
('https://screengolfers.com/usa/new-york/scarsdale', False),
('https://screengolfers.com/usa/new-york/staten-island', False),
('https://screengolfers.com/usa/new-york/westbury', False),
('https://screengolfers.com/usa/illinois/chicago', False),
('https://screengolfers.com/usa/illinois/downers-grove', False),
('https://screengolfers.com/usa/illinois/orland-park', False),
('https://screengolfers.com/usa/illinois/geneva', False),
('https://screengolfers.com/usa/illinois/schaumburg', False),
('https://screengolfers.com/usa/illinois/vernon-hills', False),
('https://screengolfers.com/usa/illinois/algonquin', False),
('https://screengolfers.com/usa/illinois/bloomington', False),
('https://screengolfers.com/usa/illinois/deerfield', False),
('https://screengolfers.com/usa/illinois/des-plaines', False),
('https://screengolfers.com/usa/illinois/elmhurst', False),
('https://screengolfers.com/usa/illinois/fairview-heights', False),
('https://screengolfers.com/usa/illinois/glen-carbon', False),
('https://screengolfers.com/usa/illinois/glenview', False),
('https://screengolfers.com/usa/illinois/kildeer', False),
('https://screengolfers.com/usa/illinois/libertyville', False),
('https://screengolfers.com/usa/illinois/naperville', False),
('https://screengolfers.com/usa/illinois/oak-brook', False),
('https://screengolfers.com/usa/illinois/rockford', False),
('https://screengolfers.com/usa/illinois/sterling', False),
('https://screengolfers.com/usa/louisiana/metairie', False),
('https://screengolfers.com/usa/louisiana/baton-rouge', False),
('https://screengolfers.com/usa/louisiana/covington', False),
('https://screengolfers.com/usa/louisiana/mandeville', False),
('https://screengolfers.com/usa/maine/augusta', False),
('https://screengolfers.com/usa/maine/biddeford', False),
('https://screengolfers.com/usa/maine/old-orchard-beach', False),
('https://screengolfers.com/usa/maine/portland', False),
('https://screengolfers.com/usa/maine/wayne', False),
('https://screengolfers.com/usa/maine/westbrook', False),
('https://screengolfers.com/usa/new-jersey/paramus', False),
('https://screengolfers.com/usa/new-jersey/east-hanover', False),
('https://screengolfers.com/usa/new-jersey/mount-laurel', False),
('https://screengolfers.com/usa/new-jersey/palisades-park', False),
('https://screengolfers.com/usa/new-jersey/clark', False),
('https://screengolfers.com/usa/new-jersey/denville', False),
('https://screengolfers.com/usa/new-jersey/east-brunswick', False),
('https://screengolfers.com/usa/new-jersey/eatontown', False),
('https://screengolfers.com/usa/new-jersey/englewood', False),
('https://screengolfers.com/usa/new-jersey/fairfield', False),
('https://screengolfers.com/usa/new-jersey/fort-lee', False),
('https://screengolfers.com/usa/new-jersey/green-brook', False),
('https://screengolfers.com/usa/new-jersey/hoboken', False),
('https://screengolfers.com/usa/new-jersey/holmdel', False),
('https://screengolfers.com/usa/new-jersey/jersey-city', False),
('https://screengolfers.com/usa/new-jersey/little-ferry', False),
('https://screengolfers.com/usa/new-jersey/lodi', False),
('https://screengolfers.com/usa/new-jersey/maple-shade', False),
('https://screengolfers.com/usa/new-jersey/maplewood', False),
('https://screengolfers.com/usa/new-jersey/middlesex', False),
('https://screengolfers.com/usa/new-jersey/northvale', False),
('https://screengolfers.com/usa/new-jersey/red-bank', False),
('https://screengolfers.com/usa/new-jersey/river-edge', False),
('https://screengolfers.com/usa/new-jersey/rochelle-park', False),
('https://screengolfers.com/usa/new-jersey/sandyston', False),
('https://screengolfers.com/usa/new-jersey/sewell', False),
('https://screengolfers.com/usa/new-jersey/sparta', False),
('https://screengolfers.com/usa/new-jersey/west-long-branch', False),
('https://screengolfers.com/usa/new-jersey/woodbridge', False),
('https://screengolfers.com/usa/north-dakota/bismarck', False),
('https://screengolfers.com/usa/north-dakota/fargo', False),
('https://screengolfers.com/usa/north-dakota/grand-forks', False),
('https://screengolfers.com/usa/north-dakota/minot', False),
('https://screengolfers.com/usa/north-dakota/west-fargo', False),
('https://screengolfers.com/usa/idaho/boise', False),
('https://screengolfers.com/usa/idaho/coeur-d-alene', False),
('https://screengolfers.com/usa/idaho/eagle', False),
('https://screengolfers.com/usa/idaho/idaho-falls', False),
('https://screengolfers.com/usa/idaho/lewiston', False),
('https://screengolfers.com/usa/idaho/meridian', False),
('https://screengolfers.com/usa/idaho/post-falls', False),
('https://screengolfers.com/usa/nebraska/omaha', False),
('https://screengolfers.com/usa/nebraska/elkhorn', False),
('https://screengolfers.com/usa/nebraska/holdrege', False),
('https://screengolfers.com/usa/nebraska/kearney', False),
('https://screengolfers.com/usa/nebraska/lincoln', False),
('https://screengolfers.com/usa/arizona/scottsdale', False),
('https://screengolfers.com/usa/arizona/chandler', False),
('https://screengolfers.com/usa/arizona/mesa', False),
('https://screengolfers.com/usa/arizona/tucson', False),
('https://screengolfers.com/usa/arizona/avondale', False),
('https://screengolfers.com/usa/arizona/glendale', False),
('https://screengolfers.com/usa/arizona/goodyear', False),
('https://screengolfers.com/usa/arizona/peoria', False),
('https://screengolfers.com/usa/arizona/phoenix', False),
('https://screengolfers.com/usa/arizona/tempe', False),
('https://screengolfers.com/usa/new-hampshire/bedford', False),
('https://screengolfers.com/usa/new-hampshire/derry', False),
('https://screengolfers.com/usa/new-hampshire/intervale', False),
('https://screengolfers.com/usa/new-hampshire/londonderry', False),
('https://screengolfers.com/usa/new-hampshire/north-hampton', False),
('https://screengolfers.com/usa/new-hampshire/portsmouth', False),
('https://screengolfers.com/usa/new-hampshire/seabrook', False),
('https://screengolfers.com/usa/new-hampshire/windham', False),
('https://screengolfers.com/usa/new-hampshire/wolfeboro', False),
('https://screengolfers.com/usa/kentucky/louisville', False),
('https://screengolfers.com/usa/kentucky/bowling-green', False),
('https://screengolfers.com/usa/kentucky/cave-city', False),
('https://screengolfers.com/usa/kentucky/eddyville', False),
('https://screengolfers.com/usa/kentucky/elizabethtown', False),
('https://screengolfers.com/usa/kentucky/lexington', False),
('https://screengolfers.com/usa/tennessee/brentwood', False),
('https://screengolfers.com/usa/tennessee/knoxville', False),
('https://screengolfers.com/usa/tennessee/murfreesboro', False),
('https://screengolfers.com/usa/tennessee/nashville', False),
('https://screengolfers.com/usa/tennessee/franklin', False),
('https://screengolfers.com/usa/tennessee/hendersonville', False),
('https://screengolfers.com/usa/tennessee/memphis', False),
('https://screengolfers.com/usa/tennessee/morrison', False),
('https://screengolfers.com/usa/tennessee/spring-hill', False),
('https://screengolfers.com/usa/washington/redmond', False),
('https://screengolfers.com/usa/washington/spokane', False),
('https://screengolfers.com/usa/washington/spokane-valley', False),
('https://screengolfers.com/usa/washington/lynnwood', False),
('https://screengolfers.com/usa/washington/seattle', False),
('https://screengolfers.com/usa/washington/vancouver', False),
('https://screengolfers.com/usa/washington/bellevue', False),
('https://screengolfers.com/usa/washington/centralia', False),
('https://screengolfers.com/usa/washington/kennewick', False),
('https://screengolfers.com/usa/washington/liberty-lake', False),
('https://screengolfers.com/usa/washington/lynden', False),
('https://screengolfers.com/usa/washington/monroe', False),
('https://screengolfers.com/usa/washington/moses-lake', False),
('https://screengolfers.com/usa/washington/oak-harbor', False),
('https://screengolfers.com/usa/washington/pullman', False),
('https://screengolfers.com/usa/washington/richland', False),
('https://screengolfers.com/usa/washington/tacoma', False),
('https://screengolfers.com/usa/washington/tukwila', False),
('https://screengolfers.com/usa/washington/tumwater', False),
('https://screengolfers.com/usa/washington/walla-walla', False),
('https://screengolfers.com/usa/utah/salt-lake-city', False),
('https://screengolfers.com/usa/utah/american-fork', False),
('https://screengolfers.com/usa/utah/farmington', False),
('https://screengolfers.com/usa/utah/holladay', False),
('https://screengolfers.com/usa/utah/lehi', False),
('https://screengolfers.com/usa/utah/ogden', False),
('https://screengolfers.com/usa/utah/riverton', False),
('https://screengolfers.com/usa/arkansas/bentonville', False),
('https://screengolfers.com/usa/arkansas/little-rock', False),
('https://screengolfers.com/usa/arkansas/conway', False),
('https://screengolfers.com/usa/arkansas/fayetteville', False),
('https://screengolfers.com/usa/arkansas/fort-smith', False),
('https://screengolfers.com/usa/california/los-angeles', False),
('https://screengolfers.com/usa/california/pasadena', False),
('https://screengolfers.com/usa/california/san-jose', False),
('https://screengolfers.com/usa/california/irvine', False),
('https://screengolfers.com/usa/california/san-diego', False),
('https://screengolfers.com/usa/california/upland', False),
('https://screengolfers.com/usa/california/brea', False),
('https://screengolfers.com/usa/california/buena-park', False),
('https://screengolfers.com/usa/california/carmel-by-the-sea', False),
('https://screengolfers.com/usa/california/colma', False),
('https://screengolfers.com/usa/california/costa-mesa', False),
('https://screengolfers.com/usa/california/danville', False),
('https://screengolfers.com/usa/california/dublin', False),
('https://screengolfers.com/usa/california/el-dorado-hills', False),
('https://screengolfers.com/usa/california/encino', False),
('https://screengolfers.com/usa/california/folsom', False),
('https://screengolfers.com/usa/california/foster-city', False),
('https://screengolfers.com/usa/california/fremont', False),
('https://screengolfers.com/usa/california/indio', False),
('https://screengolfers.com/usa/california/long-beach', False),
('https://screengolfers.com/usa/california/oxnard', False),
('https://screengolfers.com/usa/california/pleasant-hill', False),
('https://screengolfers.com/usa/california/pleasanton', False),
('https://screengolfers.com/usa/california/rancho-santa-margarita', False),
('https://screengolfers.com/usa/california/redlands', False),
('https://screengolfers.com/usa/california/roseville', False),
('https://screengolfers.com/usa/california/san-francisco', False),
('https://screengolfers.com/usa/california/santa-ana', False),
('https://screengolfers.com/usa/california/santa-monica', False),
('https://screengolfers.com/usa/california/studio-city', False),
('https://screengolfers.com/usa/california/temecula', False),
('https://screengolfers.com/usa/california/torrance', False),
('https://screengolfers.com/usa/california/walnut-creek', False),
('https://screengolfers.com/usa/iowa/cedar-rapids', False),
('https://screengolfers.com/usa/iowa/coralville', False),
('https://screengolfers.com/usa/iowa/davenport', False),
('https://screengolfers.com/usa/iowa/west-des-moines', False),
('https://screengolfers.com/usa/iowa/ames', False),
('https://screengolfers.com/usa/iowa/anamosa', False),
('https://screengolfers.com/usa/iowa/ankeny', False),
('https://screengolfers.com/usa/iowa/burlington', False),
('https://screengolfers.com/usa/iowa/carroll', False),
('https://screengolfers.com/usa/iowa/cedar-falls', False),
('https://screengolfers.com/usa/iowa/chariton', False),
('https://screengolfers.com/usa/iowa/clive', False),
('https://screengolfers.com/usa/iowa/dubuque', False),
('https://screengolfers.com/usa/iowa/fairfax', False),
('https://screengolfers.com/usa/iowa/griswold', False),
('https://screengolfers.com/usa/iowa/grundy-center', False),
('https://screengolfers.com/usa/iowa/mason-city', False),
('https://screengolfers.com/usa/iowa/orange-city', False),
('https://screengolfers.com/usa/iowa/oskaloosa', False),
('https://screengolfers.com/usa/iowa/pella', False),
('https://screengolfers.com/usa/iowa/sioux-city', False),
('https://screengolfers.com/usa/iowa/urbandale', False),
('https://screengolfers.com/usa/iowa/waukee', False),
('https://screengolfers.com/usa/iowa/waverly', False)

]

# Process each URL
for url, handle_cookies in urls:
    print(f"Processing URL: {url}")
    process_url(url, handle_cookies)

# Close the CSV file
csv_file.close()

# Close the WebDriver
driver.quit()

